# Czyszczenie danych

Przygotowanie funkcji czyszczącej.

In [11]:
def clean_string(text):

    final_string = ""

    # Make lower

    text = text.lower()

    # Remove line breaks

    text = re.sub(r'\n', '', text)

    # Remove punctuation

    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)

    # Remove stop words

    text = text.split()
    useless_words = nltk.corpus.stopwords.words("english")

    useless_words = useless_words
    text_filtered = [word for word in text if not word in useless_words]

    # Remove numbers

    text_filtered = [re.sub(r'\w*\d\w*', '', w) for w in text_filtered]

    # Lemmatize
    
    text_filtered = nlp(' '.join(text_filtered))
    text_stemmed = [y.lemma_ for y in text_filtered]

    final_string = ' '.join(text_stemmed)


    return final_string

Czyszczenie abstraktów i zapisanie ich do folderu `clean_data`

In [32]:
import os, re, string, csv, nltk, spacy
import pandas as pd
from nltk.corpus import stopwords
from IPython.display import clear_output
from tqdm import tqdm
tqdm.pandas()

nlp = spacy.load('en_core_web_sm')

paths = os.listdir('data')

for index, path in enumerate(paths):
  print(f"[{index+1}/{len(paths)}]")

  #wczytaj
  articles = pd.read_csv(f'data/{path}', sep=';', quoting=csv.QUOTE_NONE)

  #usuń puste i czyść
  articles = articles.dropna().reset_index(drop=True)

  keywords = ['tool', 'tools', 'device', 'devices', 'apparatus', 'apparatuses', 'appliance', 'appliances', 'engine', 'engines', 'gadget', 'gadgets']
  pattern = '|'.join(keywords)

  articles['abstract'] = articles['abstract'].astype(str)
  toolArticles = articles[articles['abstract'].str.contains(pattern, regex=True)].reset_index(drop=True)
  print(f"Found {len(toolArticles)} tool articles.")

  toolArticles['clean_abstract'] = toolArticles['abstract'].progress_apply(lambda x: clean_string(x))

  #dodaj
  articles = pd.DataFrame({'pmid': toolArticles['pmid'], 'clean_abstract': toolArticles['clean_abstract']})
  articles.to_csv(f'new_clean_data/clean_article{(index+1):04d}.csv', index=False)

  clear_output(wait=True)

print("Data loaded.")

Data loaded.
